# Paper-ready figures

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import plot_utils

plot_utils.configure()

Recommend (Ubuntu):
  sudo apt-get install cm-super dvipng fonts-cmu texlive-latex-extra


In [3]:
df = pd.read_json("../out/l40s-py-2026-01-17T22-56-28.jsonl", lines=True)
display(df)

,test,m,k,n,g,bits,copies,inner_reps,outer_reps,bytes_rw,ops,avg_time,avg_time_stderr,id,device,cuda_version,torch_version,triton_version,commit
0,mv_ref,1,8192,8192,64,16,100,100,100,134250496,134217728,0.000182,2.073154e-08,py-2026-01-17T22-56-28,NVIDIA L40S,12.8,2.9.1+cu128,3.5.1,5592b6dc1865162dcebc64be2a365e3ca8980ce2
1,mv,1,8192,8192,64,16,100,100,100,134250496,134217728,0.000176,1.946425e-08,py-2026-01-17T22-56-28,NVIDIA L40S,12.8,2.9.1+cu128,3.5.1,5592b6dc1865162dcebc64be2a365e3ca8980ce2
2,mm_ref,1,8192,8192,64,16,100,100,100,134250496,134217728,0.000182,2.072337e-08,py-2026-01-17T22-56-28,NVIDIA L40S,12.8,2.9.1+cu128,3.5.1,5592b6dc1865162dcebc64be2a365e3ca8980ce2
3,mm,1,8192,8192,64,16,100,100,100,134250496,134217728,0.000214,1.292943e-08,py-2026-01-17T22-56-28,NVIDIA L40S,12.8,2.9.1+cu128,3.5.1,5592b6dc1865162dcebc64be2a365e3ca8980ce2
4,mv_lut8_ref_unscaled,1,8192,8192,64,8,100,100,100,69238784,134217728,0.000090,2.303511e-09,py-2026-01-17T22-56-28,NVIDIA L40S,12.8,2.9.1+cu128,3.5.1,5592b6dc1865162dcebc64be2a365e3ca8980ce2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,mm_lut8_ref,256,3072,3072,64,2,100,100,100,5799936,4831838208,0.000081,1.035846e-07,py-2026-01-17T22-56-28,NVIDIA L40S,12.8,2.9.1+cu128,3.5.1,5592b6dc1865162dcebc64be2a365e3ca8980ce2
352,mm_lut8,256,3072,3072,64,2,100,100,100,5799936,4831838208,0.000100,1.687070e-07,py-2026-01-17T22-56-28,NVIDIA L40S,12.8,2.9.1+cu128,3.5.1,5592b6dc1865162dcebc64be2a365e3ca8980ce2
353,mm_lut8_ref_unscaled,256,3072,3072,64,1,100,100,100,4620288,4831838208,0.000658,1.787997e-09,py-2026-01-17T22-56-28,NVIDIA L40S,12.8,2.9.1+cu128,3.5.1,5592b6dc1865162dcebc64be2a365e3ca8980ce2
354,mm_lut8_ref,256,3072,3072,64,1,100,100,100,4620288,4831838208,0.000674,2.087604e-09,py-2026-01-17T22-56-28,NVIDIA L40S,12.8,2.9.1+cu128,3.5.1,5592b6dc1865162dcebc64be2a365e3ca8980ce2


In [4]:
df = pd.read_json("../out/l40s-py-2026-01-17T22-56-28.jsonl", lines=True).drop(columns=[
    "n", "g", "copies", "inner_reps", "outer_reps",
    "id", "device", "cuda_version", "torch_version", "triton_version", "commit",
])
df["gb_s"] = df["bytes_rw"] / df["avg_time"] / 1e9
df["tflops"] = df["ops"] / df["avg_time"] / 1e12

# Remove "mm" tests where "mv" was also run, for batch size 1
df = df[~((df.m == 1) & df.test.isin(["mm", "mm_ref", "mm_lut8_ref_unscaled", "mm_lut8_ref", "mm_lut8"]))]
df["kernel"] = df.test.apply(dict(
    mm="triton", mv="triton",
    mm_ref="torch", mv_ref="torch",
    mm_lut8_ref_unscaled="torch.compile:unscaled", mv_lut8_ref_unscaled="torch.compile:unscaled",
    mv_lut8_ref="torch.compile", mm_lut8_ref="torch.compile",
    mv_lut8="triton", mm_lut8="triton",
    marlin="marlin",
).__getitem__)
assert df.groupby(["m", "k", "bits", "kernel"]).avg_time.count().max() == 1, "duplicate entries with same kernel name"
df = df.drop(columns=["test"])

df.head()

,m,k,bits,bytes_rw,ops,avg_time,avg_time_stderr,gb_s,tflops,kernel
0,1,8192,16,134250496,134217728,0.000182,2.073154e-08,739.281668,0.739101,torch
1,1,8192,16,134250496,134217728,0.000176,1.946425e-08,764.350473,0.764164,triton
4,1,8192,8,69238784,134217728,0.000090,2.303511e-09,769.148813,1.490977,torch.compile:unscaled
5,1,8192,8,69238784,134217728,0.000283,3.898155e-07,244.655836,0.474259,torch.compile
6,1,8192,8,69238784,134217728,0.000096,1.730201e-08,723.931694,1.403324,triton
